In [17]:
import pandas as pd
df_index= pd.read_excel(r"datasets/For_ML_CSI300_Index_Level(2008-3-31).xlsx", engine='openpyxl')
df_index.head()


,date,index_level
0,2008-03-31,3790.53
1,2008-04-01,3582.86
2,2008-04-02,3547.98
3,2008-04-03,3650.70
4,2008-04-07,3845.82


In [18]:
df_index['date']=pd.to_datetime(df_index['date'])
df_index.set_index("date", inplace=True)
weekly_groups=df_index.groupby(pd.Grouper(freq='W'))
#find the last day of each week
df_index_weekly=weekly_groups.last()
df_index_weekly
df_index_weekly.isnull().sum()
df_index_weekly.dropna(inplace=True)

In [19]:
#add index week1 week2 week3...
import numpy as np
df_index_weekly['week'] = range(1, len(df_index_weekly) + 1)
df_index_weekly.set_index("week", inplace=True)
df_index_weekly


,index_level
week,
1,3650.70
2,3783.73
3,3272.50
4,3803.07
5,3959.12
...,...
751,3870.95
752,3998.24
753,3954.23


In [20]:
#last 52 weeks' max value for weekly_indexLevel Dataset
df_index_weekly['max_52'] = None
for i in range(52, len(df_index_weekly)+1):
    previous_52 = df_index_weekly.loc[i-52:i-1, 'index_level']  
    max_value = previous_52.max()  
    df_index_weekly.at[i, 'max_52'] = max_value  

df_index_weekly

,index_level,max_52
week,,
1,3650.70,None
2,3783.73,None
3,3272.50,None
4,3803.07,None
5,3959.12,None
...,...,...
751,3870.95,5055.12
752,3998.24,5055.12
753,3954.23,5055.12


In [21]:
#calculate ML = index_level/(max of last 52 weeks) and create a new column
df_index_weekly['ML'] = df_index_weekly['index_level']/df_index_weekly['max_52']
df_index_weekly

,index_level,max_52,ML
week,,,
1,3650.70,None,NaN
2,3783.73,None,NaN
3,3272.50,None,NaN
4,3803.07,None,NaN
5,3959.12,None,NaN
...,...,...,...
751,3870.95,5055.12,0.765748
752,3998.24,5055.12,0.790929
753,3954.23,5055.12,0.782223


In [23]:
#find the mean and standard deviation of ML for past 52 weeks
df_index_weekly['mean_52'] = None
df_index_weekly['std_52'] = None
for i in range(104, len(df_index_weekly)+1):
    previous_52 = df_index_weekly.loc[i-52:i-1, 'ML']  
    mean_value = previous_52.mean()  
    std_value = previous_52.std()  
    df_index_weekly.at[i, 'mean_52'] = mean_value  
    df_index_weekly.at[i, 'std_52'] = std_value

df_index_weekly

,index_level,max_52,ML,mean_52,std_52
week,,,,,
1,3650.70,None,NaN,None,None
2,3783.73,None,NaN,None,None
3,3272.50,None,NaN,None,None
4,3803.07,None,NaN,None,None
5,3959.12,None,NaN,None,None
...,...,...,...,...,...
751,3870.95,5055.12,0.765748,0.798755,0.039911
752,3998.24,5055.12,0.790929,0.797289,0.039692
753,3954.23,5055.12,0.782223,0.796406,0.039303


In [25]:
#df_index_weekly['mean-2.5*std'] = df_index_weekly['mean_52'] - 2.5*df_index_weekly['std_52']
df_index_weekly['mean-2.5*std'] = df_index_weekly['mean_52'] - 1.5*df_index_weekly['std_52']
df_index_weekly['CC'] = np.where(df_index_weekly['ML'] < df_index_weekly['mean-2.5*std'], 1, 0)
df_index_weekly

,index_level,max_52,ML,mean_52,std_52,mean-2.5*std,CC
week,,,,,,,
1,3650.70,None,NaN,None,None,NaN,0
2,3783.73,None,NaN,None,None,NaN,0
3,3272.50,None,NaN,None,None,NaN,0
4,3803.07,None,NaN,None,None,NaN,0
5,3959.12,None,NaN,None,None,NaN,0
...,...,...,...,...,...,...,...
751,3870.95,5055.12,0.765748,0.798755,0.039911,0.738888,0
752,3998.24,5055.12,0.790929,0.797289,0.039692,0.737751,0
753,3954.23,5055.12,0.782223,0.796406,0.039303,0.737451,0


In [26]:
#to csv
df_index_weekly.to_csv(r'datasets\CSI300_IL_Weekly_ML.csv', index = True, header=True)

In [27]:
df_index_weekly.describe()

,index_level,CC
count,755.000000,755.000000
mean,3389.810490,0.162914
std,851.270796,0.369532
min,1663.660000,0.000000
25%,2647.560000,0.000000
50%,3340.130000,0.000000
75%,3904.000000,0.000000
max,5807.720000,1.000000
